In [1]:
import duckdb

In [2]:
conn = duckdb.connect(database='data_duckdb.db', read_only=False)

In [3]:
df = conn.execute("""
                  SELECT *
                  FROM(SELECT *, ROW_NUMBER() OVER (PARTITION BY NATBR ORDER BY date_ingestion DESC) AS row 
                  FROM bronze_z0019)
                  WHERE row=1""").fetchdf()
df.head(10)

,NATBR,MAKTX,WERKS,MAINS,LABST,name_file,date_ingestion,row
0,10003,PREGO,BT10,100,50,z0019_1.csv,2025-04-20 09:20:09.654996,1
1,10006,PREGO,BT10,100,60,z0019_2.csv,2025-04-20 09:21:33.421997,1
2,10001,PARAFUSO,BT10,100,100,z0019_1.csv,2025-04-20 09:20:09.654996,1
3,10004,SERRA,BT50,100,200,z0019_2.csv,2025-04-20 09:21:33.421997,1
4,10002,MARTELO,BT50,100,1500,z0019_1.csv,2025-04-20 09:20:09.654996,1
5,10005,MACHADO,BT50,100,100,z0019_2.csv,2025-04-20 09:21:33.421997,1


In [4]:
df_final = df.drop(columns=['name_file', 'date_ingestion', 'row'])

df_final = df_final.rename(columns={
    'NATBR': 'id',
    'MAKTX': 'product_nm',
    'WERKS': 'category_id',
    'MAINS': 'vendor_id',
    'LABST': 'price'
})

df_final.head()

,id,product_nm,category_id,vendor_id,price
0,10003,PREGO,BT10,100,50
1,10006,PREGO,BT10,100,60
2,10001,PARAFUSO,BT10,100,100
3,10004,SERRA,BT50,100,200
4,10002,MARTELO,BT50,100,1500


In [5]:
df2 = df_final.copy()

In [8]:
df2.dtypes

id             object
product_nm     object
category_id    object
vendor_id      object
price          object
dtype: object

In [9]:
df2 = df2.astype(
    {
        'id': int,
        'product_nm' : str,
        'category_id' : str,
        'vendor_id' : int,
        'price' : float
    }
)

In [10]:
df2.dtypes

id               int64
product_nm      object
category_id     object
vendor_id        int64
price          float64
dtype: object

In [11]:
conn.execute("""
CREATE TABLE IF NOT EXISTS products(
             id BIGINT,
             product_nm TEXT,
             category_id TEXT,
             vendor_id BIGINT,
             price FLOAT
             )
             """)

In [12]:
conn.execute("INSERT INTO products SELECT * FROM df2")

In [15]:
df_result = conn.execute("SELECT * FROM products").fetchdf()
df_result.head()

,id,product_nm,category_id,vendor_id,price
0,10003,PREGO,BT10,100,50.0
1,10006,PREGO,BT10,100,60.0
2,10001,PARAFUSO,BT10,100,100.0
3,10004,SERRA,BT50,100,200.0
4,10002,MARTELO,BT50,100,1500.0


In [16]:
conn.close()